In [ ]:
import pandas as pd
import altair as alt

In [ ]:
read_depth = '/Users/ivan/Desktop/CRISPResso_dev_outputs/20250123_BARD1_NormReadDepth.tsv'
gsp_input_file = '/Users/ivan/Desktop/CRISPResso_dev_outputs/deletion_inputs.xlsx'

In [ ]:
def read_input(file):
    input_params = pd.read_excel(file) #Reads input file


    return input_params

In [ ]:
def read_read_depth(file,input_params):
    raw_df = pd.read_csv(file, sep = '\t')

    raw_df = raw_df.loc[raw_df['day'].isin(['D13'])]

    targets = input_params['target'].tolist()

    df = raw_df.loc[raw_df['target'].isin(targets)]


    i = 0
    cds = []
    while i < len(input_params):
        cds_start = input_params['coding_start'][i]
        cds_end = input_params['coding_end'][i]
        for j in range(cds_start, cds_end + 1):
            if j not in cds:
                cds.append(j)
            
        i += 1


    df = df.loc[df['pos'].isin(cds)]
    df = df.reset_index(drop = True)
    
    return df

In [ ]:
def process_read_depth(df):

    grouped = df.groupby('pos')

    median_depth = df.groupby('pos').agg({
    'normdepth': 'median',
    'target': 'first'
    }).reset_index()

    print(median_depth)
    
    cds_pos = []

    for i in range(len(median_depth)):
        cds_pos.append(i+1)

    cds_pos = cds_pos[::-1]
    #median_depth = median_depth.drop(columns = ['offset'])

    median_depth['CDSpos'] = cds_pos
    median_depth = median_depth.rename(columns = {'normdepth': 'median_depth'})

    
    print(median_depth)
    return median_depth

In [ ]:
def graph_median_depth(df):

    plot =  alt.Chart(df).mark_line(point = True).encode(
        x = alt.X('CDSpos', axis = alt.Axis(title = 'CDS Position')),
        y = alt.Y('median_depth', axis = alt.Axis(title = 'Median Normalized Depth')),
        color = alt.Color('target', legend = alt.Legend(title = 'SGE Region')),
        tooltip = [alt.Tooltip('target', title = 'SGE Target: '),
                   alt.Tooltip('CDSpos', title = 'CDS Position: '),
                   alt.Tooltip('median_depth', title = 'Median Normalized Depth: ')]
    ).properties(
        width = 600, 
        height = 400,
    ).interactive()
    
    plot.display()

In [ ]:
def main():
    input_params = read_input(gsp_input_file)
    all_reps_depth = read_read_depth(read_depth,input_params)
    collapsed_depth = process_read_depth(all_reps_depth)
    graph_median_depth(collapsed_depth)

In [ ]:
main()